In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import datetime
import random
from statsmodels.stats.power import tt_ind_solve_power

def date_converter(x):
    return datetime.datetime.strptime(x,'%Y-%m-%d')

### Задание 3
**Срок сдачи: 07 апреля 18:00**

В этом задание вы будете работать с выручкой - метрикой с достаточно высокой дисперсией и соответсвенно низкой чувствительностью к изменениям.

Вам предстоит разработать подход по повышению чувствительности этой метрики.
От успеха данной инициативы зависит то, как быстро мы сможем проверять продуктовые гипотезы.

#### Для планирования эксперимента изучите данные и метрики 2 балла:

- Загрузите файл и изучите данные;
- Изучите основные статистики метрики: среднее, стандартное отклонение;
- Изучите то, как эти статистики изменяются внутри срезов: user_segment, region, category

#### Предскажите минимально детектируемый эффект (MDE) 2 балла:

- Для размеров тестовых групп 10,25,50% и длительностей 30, 60, 90 дней предскажите минимально детектируемый эффект;


#### CUPED 6 баллов
Разработайте подход по снижению дисперсии на основе методики CUPED:

- Рассчитайте CUPED метрику на основе данных за периода в 30 дней перед предполагаемым периодом сбора основной выборки и опишите как изменился MDE для выборок 50/50%, собранных за 30 дней (2 бала). 
- Проверьте насколько данный подход подвержен сезональности. Как меняется MDE, если применять данный подход в разные периоды (2 бала). 
- Оптимизируйте подход варьируя количество дней до теста, которые вы будете для расчёта CUPED метрики. Какое максимальное снижение MDE вы смогли достичь (2 бала).



#### * Стратификация 4 балла (задание для получения дополнительных баллов)
Разработайте подход по снижению дисперсии на основе методики стратификации выборок:

- Стратифицируйте выборки внутри срезов user_segment, region, category
- Рассчитайте стратифицированные среднее,стандартное отклонение для этих выборок и MDE для теста 50/50% длительностью 30 дней.
- В каких срезах достигается наибольшее снижение MDE?



#### Описание данныx

event_date - дата 

user_id - идентификатор пользователя

user_segment - сегмент пользователей

region - регион

category - категория авито

revenue_amount - сумма покупок


In [69]:
try:
    filename = 'user_amount_var_reduction.csv'
    df = pd.read_csv(filename)
    df['event_date'] = pd.to_datetime(df['event_date']).dt.date
except FileNotFoundError:
    print(f'Все же нужно скачать большой файлик {filename}')

Все же нужно скачать большой файлик user_amount_var_reduction.csv


In [2]:
PARQUET_NAME = 'hw3_aaa.parquet'

In [ ]:
try:
    df.to_parquet(PARQUET_NAME)
except Exception as e:
    print('Необходимо скачать файл user_amount_var_reduction.csv,'
         ' считать его csv, а только лишь затем записать parquet.'
         ' Используется для ускорения считывания')

In [3]:
df = pd.read_parquet(PARQUET_NAME)
df['event_date'] = pd.to_datetime(df['event_date']).dt.date

In [5]:
df['revenue_amount'].describe()

count    8.625842e+06
mean     4.009047e+02
std      9.602217e+02
min      1.000000e+00
25%      8.800000e+01
50%      1.670000e+02
75%      4.050000e+02
max      2.881230e+05
Name: revenue_amount, dtype: float64

Посмотрим на данные в разрезах сегментов, регионов и т.д.

In [8]:
df.columns

Index(['event_date', 'user_id', 'user_segment', 'category', 'region',
       'revenue_amount'],
      dtype='object')

In [9]:
df['user_segment'].unique()

array([4626944681007198896, 2376074197230840906,  853431062533847667],
      dtype=int64)

In [10]:
df.groupby('user_segment')['revenue_amount'].describe()

,count,mean,std,min,25%,50%,75%,max
user_segment,,,,,,,,
853431062533847667,1001507.0,1094.330305,2392.330098,1.0,226.0,489.0,1113.0,288123.0
2376074197230840906,4013951.0,401.615956,571.446013,4.0,109.0,209.0,475.0,52540.0
4626944681007198896,3610384.0,207.760172,285.545686,4.0,63.0,112.0,230.0,19950.0


Скорее всего какие-то вариации mass, maffluent, affluent сегменты, разве что mass слишком мало, а affluent слишком много по количеству. Тем ни менее видим, что наши сегменты хорошо разделяются по перцентилям или по максимуму, например.

In [28]:
df.groupby('region')['revenue_amount'].describe().sort_values('count', ascending=False)

,count,mean,std,min,25%,50%,75%,max
region,,,,,,,,
1481836092404435976,942345.0,709.130762,1819.635279,6.0,139.00,280.0,685.0,288123.0
3855810523611026650,704283.0,388.279534,813.357258,4.0,92.00,168.0,419.0,140792.0
4848067518890897757,530838.0,644.019141,1407.455008,8.0,133.00,279.0,650.0,141891.0
5780543780372929118,395734.0,536.875280,1324.789564,7.0,118.00,230.0,552.0,202737.0
8867014108346120905,305261.0,445.188098,712.591963,6.0,111.00,209.0,489.0,41006.0
...,...,...,...,...,...,...,...,...
9173615405044165275,3073.0,189.498536,230.607352,6.0,55.00,124.0,196.0,3538.0
3528733747751745134,1771.0,265.359119,377.548298,8.0,75.00,133.0,304.0,4946.0
405364780634206711,1040.0,205.209615,295.197711,8.0,63.75,112.0,231.0,5121.0


Тут сложно что-то сказать, единственное, что регионов 84, что подозрительно похоже на какую-то часть РФ.

In [14]:
df.groupby('category')['revenue_amount'].describe().sort_values('count', ascending=False)

,count,mean,std,min,25%,50%,75%,max
category,,,,,,,,
3187769798308634693,767177.0,467.654020,790.312625,11.0,118.0,244.0,524.0,47314.0
7204754148424990691,736535.0,352.174948,511.821260,4.0,90.0,188.0,405.0,36824.0
6549025562984299367,725571.0,410.800302,1899.114650,20.0,69.0,139.0,279.0,288123.0
7948270325129383019,698731.0,792.677807,1352.172272,11.0,209.0,419.0,906.0,110212.0
57514010642945023,689886.0,347.826167,605.864274,20.0,109.0,167.0,369.0,42271.0
1472231361867825698,627129.0,258.353200,594.984151,4.0,56.0,112.0,231.0,30429.0
4394253463123676325,580595.0,136.595029,300.843418,6.0,39.0,70.0,140.0,46074.0
5983323600580891431,480199.0,433.565037,535.078480,11.0,118.0,241.0,552.0,25226.0
174490086998335078,379803.0,325.149759,764.813512,1.0,62.0,132.0,291.0,51403.0


In [15]:
df.groupby(['region', 'category'])['revenue_amount'].describe().sort_values('count', 
                                                                           ascending=False)

count         mean          std  \
region              category                                                  
1481836092404435976 4394253463123676325  114708.0   262.122895   572.494605   
                    1472231361867825698  113672.0   378.266046   741.820649   
                    7948270325129383019   87635.0  1460.587870  2447.247208   
                    7204754148424990691   87557.0   570.463938   683.008500   
3855810523611026650 57514010642945023     79322.0   302.059303   403.054701   
...                                           ...          ...          ...   
7994151639322831196 5858189596811644833       1.0    69.000000          NaN   
928013360687874809  1560592244484430230       1.0   139.000000          NaN   
                    3358911609809004109       1.0    90.000000          NaN   
7198416805188763410 5858189596811644833       1.0   244.000000          NaN   
5021831474948584079 5858189596811644833       1.0   419.000000          NaN   

                                           min    25%    50%     75%       max  
region              category                                                    
1481836092404435976 4394253463123676325    8.0   70.0  140.0   280.0   46074.0  
                    1472231361867825698    8.0   98.0  161.0   392.0   30135.0  
                    7948270325129383019   34.0  328.0  979.0  1741.0  110212.0  
                    7204754148424990691   13.0  139.0  328.0   685.0   13483.0  
3855810523611026650 57514010642945023     24.0  109.0  159.0   335.0   11588.0  
...                                        ...    ...    ...     ...       ...  
7994151639322831196 5858189596811644833   69.0   69.0   69.0    69.0      69.0  
928013360687874809  1560592244484430230  139.0  139.0  139.0   139.0     139.0  
                    3358911609809004109   90.0   90.0   90.0    90.0      90.0  
7198416805188763410 5858189596811644833  244.0  244.0  244.0   244.0     244.0  
5021831474948584079 5858189596811644833  419.0  419.0  419.0   419.0     419.0  

[2736 rows x 8 columns]

In [17]:
df.groupby(['user_segment', 'category'])['revenue_amount'].describe().sort_values('count', 
                                                                           ascending=False)

count        mean         std  \
user_segment        category                                                
2376074197230840906 3187769798308634693  521240.0  430.776074  542.002105   
                    57514010642945023    519613.0  317.388283  438.741864   
4626944681007198896 4394253463123676325  457493.0  112.062056  161.413000   
2376074197230840906 7948270325129383019  446125.0  671.103381  815.941148   
                    7204754148424990691  430044.0  391.471849  547.621606   
...                                           ...         ...         ...   
853431062533847667  4658595883120885993     308.0  866.246753  875.782370   
2376074197230840906 7164285449278522823     282.0  853.351064  928.982036   
4626944681007198896 2343719193625129042     141.0  405.170213  372.672720   
                    7164285449278522823      35.0  273.200000  409.894305   
853431062533847667  5387097547938806535      12.0  197.333333  101.673572   

                                          min    25%    50%      75%      max  
user_segment        category                                                   
2376074197230840906 3187769798308634693  11.0  119.0  251.0   524.00  19594.0  
                    57514010642945023    20.0  109.0  167.0   352.00  21496.0  
4626944681007198896 4394253463123676325   6.0   35.0   70.0   126.00  19068.0  
2376074197230840906 7948270325129383019  11.0  209.0  377.0   823.00  52540.0  
                    7204754148424990691   4.0  109.0  209.0   454.00  36824.0  
...                                       ...    ...    ...      ...      ...  
853431062533847667  4658595883120885993  55.0  350.0  714.0  1029.00   5250.0  
2376074197230840906 7164285449278522823  48.0  244.0  527.0  1099.50   4722.0  
4626944681007198896 2343719193625129042  48.0  139.0  294.0   517.00   1952.0  
                    7164285449278522823  21.0   69.0  139.0   254.50   1952.0  
853431062533847667  5387097547938806535  20.0  139.0  174.0   266.75    377.0  

[96 rows x 8 columns]

In [18]:
df.groupby(['user_segment', 'region'])['revenue_amount'].describe().sort_values('count', 
                                                                           ascending=False)

count        mean         std  \
user_segment        region                                                  
4626944681007198896 1481836092404435976  429737.0  312.092135  412.864361   
2376074197230840906 1481836092404435976  384382.0  664.102619  873.090407   
                    3855810523611026650  357095.0  396.760316  537.433703   
4626944681007198896 3855810523611026650  283202.0  223.714250  294.779461   
2376074197230840906 4848067518890897757  237960.0  594.217360  786.546091   
...                                           ...         ...         ...   
4626944681007198896 928013360687874809      146.0  154.616438  150.736823   
2376074197230840906 8666105090637809898      95.0  332.189474  356.214861   
                    928013360687874809       57.0  267.421053  250.907421   
853431062533847667  8666105090637809898      37.0  196.702703  210.436433   
                    928013360687874809       14.0  312.000000  612.041602   

                                          min    25%    50%    75%      max  
user_segment        region                                                   
4626944681007198896 1481836092404435976   6.0   91.0  161.0  335.0  19950.0  
2376074197230840906 1481836092404435976   8.0  167.0  350.0  832.0  52540.0  
                    3855810523611026650   4.0  111.0  209.0  475.0  15884.0  
4626944681007198896 3855810523611026650   4.0   64.0  126.0  251.0   9762.0  
2376074197230840906 4848067518890897757  11.0  167.0  329.0  700.0  36824.0  
...                                       ...    ...    ...    ...      ...  
4626944681007198896 928013360687874809   14.0   63.0  110.0  189.0    923.0  
2376074197230840906 8666105090637809898  27.0   90.0  196.0  397.0   1504.0  
                    928013360687874809   13.0  104.0  188.0  377.0   1280.0  
853431062533847667  8666105090637809898  41.0   69.0  132.0  209.0   1280.0  
                    928013360687874809   34.0   69.0  135.5  261.5   2413.0  

[252 rows x 8 columns]

In [19]:
df.groupby(['user_segment', 'region', 'category'])['revenue_amount'].describe().sort_values('count', 
                                                                           ascending=False)

count  \
user_segment        region              category                       
4626944681007198896 1481836092404435976 4394253463123676325  79402.0   
                                        1472231361867825698  75832.0   
2376074197230840906 1481836092404435976 7948270325129383019  56076.0   
                    3855810523611026650 57514010642945023    55579.0   
                    1481836092404435976 7204754148424990691  46696.0   
...                                                              ...   
853431062533847667  2510779536173805371 5983323600580891431      1.0   
                    7887606166042135504 1472231361867825698      1.0   
                    2510779536173805371 5284347926600386297      1.0   
4626944681007198896 3413200001141702850 2343719193625129042      1.0   
853431062533847667  7000315305736052047 3358911609809004109      1.0   

                                                                    mean  \
user_segment        region              category                           
4626944681007198896 1481836092404435976 4394253463123676325   206.077139   
                                        1472231361867825698   241.364253   
2376074197230840906 1481836092404435976 7948270325129383019  1016.042728   
                    3855810523611026650 57514010642945023     309.764659   
                    1481836092404435976 7204754148424990691   653.571526   
...                                                                  ...   
853431062533847667  2510779536173805371 5983323600580891431   293.000000   
                    7887606166042135504 1472231361867825698   112.000000   
                    2510779536173805371 5284347926600386297    70.000000   
4626944681007198896 3413200001141702850 2343719193625129042   118.000000   
853431062533847667  7000315305736052047 3358911609809004109    55.000000   

                                                                     std  \
user_segment        region              category                           
4626944681007198896 1481836092404435976 4394253463123676325   291.822025   
                                        1472231361867825698   290.114483   
2376074197230840906 1481836092404435976 7948270325129383019  1189.439377   
                    3855810523611026650 57514010642945023     416.709767   
                    1481836092404435976 7204754148424990691   762.268211   
...                                                                  ...   
853431062533847667  2510779536173805371 5983323600580891431          NaN   
                    7887606166042135504 1472231361867825698          NaN   
                    2510779536173805371 5284347926600386297          NaN   
4626944681007198896 3413200001141702850 2343719193625129042          NaN   
853431062533847667  7000315305736052047 3358911609809004109          NaN   

                                                               min    25%  \
user_segment        region              category                            
4626944681007198896 1481836092404435976 4394253463123676325    8.0   70.0   
                                        1472231361867825698    8.0   91.0   
2376074197230840906 1481836092404435976 7948270325129383019   34.0  328.0   
                    3855810523611026650 57514010642945023     24.0  109.0   
                    1481836092404435976 7204754148424990691   14.0  160.0   
...                                                            ...    ...   
853431062533847667  2510779536173805371 5983323600580891431  293.0  293.0   
                    7887606166042135504 1472231361867825698  112.0  112.0   
                    2510779536173805371 5284347926600386297   70.0   70.0   
4626944681007198896 3413200001141702850 2343719193625129042  118.0  118.0   
853431062533847667  7000315305736052047 3358911609809004109   55.0   55.0   

                                                               50%     75%  \
user_segment        region              category                    

## MDE

In [14]:
df.columns

Index(['event_date', 'user_id', 'user_segment', 'category', 'region',
       'revenue_amount'],
      dtype='object')

In [30]:
from statsmodels.stats.power import tt_ind_solve_power

In [321]:
def choose_date(data, max_date, delta, share, offset): 
    max_date -= pd.Timedelta(days=offset)  # сместим максимальную дату
    start_date = max_date - pd.Timedelta(days=delta)
    tmp = data[(data.event_date>=start_date)&(data.event_date<=max_date)].copy()
    tmp=tmp.groupby(['user_id']).sum().reset_index()
    #tmp['id'] = range(tmp.shape[0])  # хэширование
    x = tmp[(tmp.user_id)%100<=100*share].reset_index()
    return x

In [237]:
def get_mde(data: pd.DataFrame,
            share: float,
            shift_days: int,
            alpha: float=0.01,
            power: float=0.8,
            offset: int=45,
            metric: str='revenue_amount'
            ) -> float:
    """
    Возвращает минимально детектируемый эффект в %
    Ожидается наличие библиотеки statmodels
    args:
        :param: data - наш датасет, в котором ожидаются
        колонки ['event_date'] + [metric]
        :param: share - доля в одной выборке
        :param: shift_days - на сколько дней сместить 
        относительно максимальной даты (смещенной)
        :param: alpha - уровень значимости, он же FPR 
        :param: offset - на сколько сместиться от максимальной даты
        :param: metric - колонка, в которой находится значения метрики
        По умолчанию считаем для revenue_amount
    Return:
        долю минимально детектируемого эффекта, для получения
        процентов нужно домножить на 100.
    """
    from statsmodels.stats.power import tt_ind_solve_power
    
    max_date = data['event_date'].max()
    df_filtered = choose_date(data, max_date, shift_days, share, offset)
    mean = df_filtered[metric].mean()
    std = df_filtered[metric].std()
    effect_size = tt_ind_solve_power(effect_size=None, alpha=alpha, power=power, nobs1=df_filtered.shape[0],
                                      ratio=1, alternative = 'two-sided')
    
    return effect_size * (std / mean)

In [238]:
test_shares = [0.1, 0.25, 0.5]
count_days = [30, 60, 90]

In [239]:
import warnings
warnings.filterwarnings('ignore')

In [240]:
for share in test_shares:
    for shift in count_days:
        mde = get_mde(data=df, 
                      share=share,
                      shift_days=shift
                    )  # power = 0.8 and alpha = 0.01 by default
        print(f'Для доли теста {int(share * 100)}% и количества дней {shift} MDE = {mde * 100:.3f}%')

Для доли теста 10% и количества дней 30 MDE = 16.655%
Для доли теста 10% и количества дней 60 MDE = 13.674%
Для доли теста 10% и количества дней 90 MDE = 11.373%
Для доли теста 25% и количества дней 30 MDE = 9.322%
Для доли теста 25% и количества дней 60 MDE = 9.111%
Для доли теста 25% и количества дней 90 MDE = 7.651%
Для доли теста 50% и количества дней 30 MDE = 6.202%
Для доли теста 50% и количества дней 60 MDE = 6.012%
Для доли теста 50% и количества дней 90 MDE = 5.253%


Видим, что в целом имеем плохие MDE, которые мы можем задетектировать, поэтому хотим занизить через понижение дисперсии

# Cuped

Алгоритм:
Возьмем нужный нам интервал времени, затем отойдем от него на 1 месяц (30 дней), найдем для user_id агрегат выручки (sum), сделаем left join нашего интервала на полученное pre_revenue. Сделаем заполнение нулями тех, где нет данных.

In [241]:
max_date = df['event_date'].max()

In [242]:
max_date

datetime.date(2019, 12, 9)

In [243]:
CONST_SHIFT_DAYS = 30

In [244]:
assert (pd.to_datetime(max_date) - pd.to_datetime('2019-11-09')).days == CONST_SHIFT_DAYS  # 30 дней

In [245]:
df_expr = df[(df['event_date'] <= max_date) &
            (df['event_date'] >= pd.to_datetime('2019-11-09'))]

In [246]:
pd.to_datetime(max_date) - pd.Timedelta(days=CONST_SHIFT_DAYS)

Timestamp('2019-11-09 00:00:00')

In [247]:
pd.to_datetime('2019-11-09') - pd.Timedelta(days=CONST_SHIFT_DAYS+1)

Timestamp('2019-10-09 00:00:00')

In [248]:
pre_filter_dates = (df['event_date'] < pd.to_datetime(max_date) - pd.Timedelta(days=CONST_SHIFT_DAYS)) &\
                    (df['event_date'] >= pd.to_datetime('2019-11-09') - pd.Timedelta(days=CONST_SHIFT_DAYS+1))

In [249]:
df_pre = df[pre_filter_dates][['user_id', 'event_date', 'revenue_amount']]

In [250]:
assert (df_pre['event_date'].max() - df_pre['event_date'].min()).days == 30, \
 'Что-то не так с окном в 30 дней в пре-тесте, нужно проверить!'

In [251]:
import gc

In [252]:
#df_pre_metric = df_pre.groupby('user_id').agg(pre_revenue=('revenue_amount', 'sum')).reset_index()
df_pre_metric = df_pre.groupby('user_id').agg(pre_revenue=('revenue_amount', 'mean')).reset_index()  # mean лучше - дальше
del df_pre
gc.collect()
df_pre_metric.head()

,user_id,pre_revenue
0,4358074930207,167.000000
1,9257530993551,90.000000
2,23708868092799,46.250000
3,39239811475386,234.500000
4,51991933272698,512.666667


In [253]:
df_expr = df_expr.merge(df_pre_metric, how='left', on='user_id')

In [254]:
df_expr['pre_revenue'].isna().mean()

0.31719995056863526

Как-то не очень получилось, у нас слишком много нулей, нужно сделать sanity тест на ошибочность операций.

In [255]:
df_expr[df_expr['pre_revenue'].isna()].sample(4)

,event_date,user_id,user_segment,category,region,revenue_amount,pre_revenue
1146505,2019-12-03,1994773331688371922,4626944681007198896,3358911609809004109,4459452014453832958,196,NaN
1186393,2019-12-02,5260868313786886059,4626944681007198896,3187769798308634693,1116465816370945844,454,NaN
951869,2019-12-04,5487925348047550942,2376074197230840906,4394253463123676325,5106431960058622523,35,NaN
1073410,2019-12-03,6932930756292975518,4626944681007198896,1472231361867825698,6690314761018840941,21,NaN


In [256]:
absent_user_ids = df_expr[df_expr['pre_revenue'].isna()]['user_id'].unique().tolist()

In [257]:
assert df[pre_filter_dates & (df['user_id'].isin(absent_user_ids))].shape[0] == 0

Выглядит так, как будто я не ошибся. Как говорили великие: "У меня был план, и я его придерживался" - заполним нулями пропуски.

In [258]:
df_expr['pre_revenue'].fillna(0, inplace=True)

In [259]:
assert df_expr['pre_revenue'].isna().mean() == 0.

In [260]:
teta = np.cov(df_expr.revenue_amount,
              df_expr.pre_revenue, ddof=1)[1,0]/np.var(df_expr.pre_revenue, ddof=1)

In [261]:
corr = np.corrcoef(df_expr.revenue_amount,df_expr.pre_revenue)[1,0] 

In [162]:
corr

0.31812351151893675

Ну так себе, конечно, хотелось бы бОльшую корреляцию. Возможно смена агрегата может помочь. Поменяем агрегат на среднее.

In [263]:
corr

0.6646426263367151

Такая корреляция уже лучше!

In [264]:
mean_pre_revenue = df_expr['pre_revenue'].mean()
df_expr['cuped_revenue'] = df_expr['revenue_amount'] - teta * (df_expr['pre_revenue'] - mean_pre_revenue) 

Проверим MDE уже такой метрики, ожидаем увидить меньшее значение.

In [265]:
round(get_mde(df_expr,
              share=0.5, 
              shift_days=CONST_SHIFT_DAYS,
              alpha=0.01,
              power=0.8,
              offset=0,
              metric='cuped_revenue'
       ) * 100, 3)

2.78

То есть в районе 2.8% для выборки 50% в тест и сроком 30 дней. Посмотрим, на сколько же успешна наша cuped для увеличения чувствительности метрики выручки.

In [266]:
params = {
    'data': df_expr,
    'share': 0.5,
    'shift_days': CONST_SHIFT_DAYS,
    'alpha': 0.01,
    'power': 0.8,
    'offset': 0
}

In [267]:
result = 1 - get_mde(**params, metric='cuped_revenue') / get_mde(**params, metric='revenue_amount')

In [268]:
print(f'Мы уменьшили MDE на {round(result * 100, 1)}%!')

Мы уменьшили MDE на 53.8%!


Крайне неплохо! улучшили чувствительность теста на ~53.8%, используя исторические данные. Теперь требуется найти гиперпараметры размера окна смещения, чтобы найти наибольшее усиление чувствительности. Сначала посмотрим сезонность.

In [269]:
from typing import Tuple

In [270]:
def preprocess_data(df: pd.DataFrame,
                    offset: int=CONST_SHIFT_DAYS,
                    pre_test_offset: int=CONST_SHIFT_DAYS,
                    move_test: bool = True,
                    test_duration: int=30
                   ) -> Tuple[int, pd.DataFrame]:
    """
    Вспомогательная функция, которая сделает препроцессинг свыше
    с отличием в том, что управляем сдвигом по дате для фильтров
    Т.е. разделит на тест и претест, создаст агрегат по user_id
    Сделает джоин, заполнит нулями пропуски. Посчитает cuped метрику
    args:
        :param: df - датасет для которого требуется сделать препроцессинг
        :param: offset - сдвиг, который требуется добавить\
        :param: pre_test_offset - параметр, позволяющий сдвинуть 
        :param: move_test - Нужно ли сдвигать время теста
        :param: test_duration - Продолжнительность теста
    Return:
        кортеж из датасета с колонками ['event_date', 'revenue_amount',
                                        'cuped_revenue']
        и текущего реферсного месяца
    """
    CONST_REFERENCE_DATE = pd.to_datetime('2019-12-09')
    CONST_SHIFT_DAYS = test_duration
    if move_test:
        window_upper = CONST_REFERENCE_DATE - pd.Timedelta(days=offset)
    else: 
        window_upper = CONST_REFERENCE_DATE
    window_lower = window_upper - pd.Timedelta(days=CONST_SHIFT_DAYS)  # смотрим интервал в 30 дней всегда
    df_expr = df[(df['event_date'] <= window_upper) &
            (df['event_date'] >= window_lower)]
    shift_pretest = CONST_SHIFT_DAYS  # если мы двигаем тест, то тогда константа 
    if not move_test:  # не двигаем тест, то нужно сдвинуть претест, иначе будут одинаковые значения
        shift_pretest += offset
    pretest_start = window_lower - pd.Timedelta(days=shift_pretest+1+pre_test_offset)
    pretest_end = window_upper - pd.Timedelta(days=shift_pretest)    
    pre_filter_dates = (df['event_date'] < pretest_end) &\
                    (df['event_date'] >= pretest_start) # prevent leaks in data
    df_pre = df[pre_filter_dates][['user_id', 'revenue_amount', 'event_date']]
    df_pre_metric = df_pre.groupby('user_id').agg(pre_revenue=('revenue_amount', 'mean'))\
                          .reset_index()
    del df_pre 
    gc.collect()
    df_expr = df_expr.merge(df_pre_metric, how='left', on='user_id')
    df_expr['pre_revenue'].fillna(0, inplace=True)
    assert df_expr['pre_revenue'].isna().mean() == 0., 'Na fill havent worked! Check'
    teta = np.cov(df_expr.revenue_amount,
              df_expr.pre_revenue, ddof=1)[1,0]/np.var(df_expr.pre_revenue, ddof=1)
    corr = np.corrcoef(df_expr.revenue_amount,df_expr.pre_revenue)[1,0] 
    pre_revenue_mean = df_expr['pre_revenue'].mean()
    df_expr['cuped_revenue'] = df_expr['revenue_amount'] - teta * (df_expr['pre_revenue'] - pre_revenue_mean)
    return (df_expr,
            window_lower.strftime('%Y-%m-%d'),
            pretest_start.strftime('%Y-%m-%d'),
            pretest_end.strftime('%Y-%m-%d') 
           )

In [271]:
(max_date - df['event_date'].min()).days

174

Пробежимся по разным месяцам и попробуем посмотреть, на сколько дисперсия уменьшается, разницу между тестом и пре-тестом будем смотреть в один месяц.

In [272]:
result_dict = {}
for offset in [0, 30, 60, 90, 120]:
    df_expr_offset, expr_start, pretest_start, pretest_end  = preprocess_data(df, offset=offset, pre_test_offset=0)
    params = {
        'data': df_expr_offset,
        'share': 0.5,
        'shift_days': CONST_SHIFT_DAYS,
        'alpha': 0.01,
        'power': 0.8,
        'offset': 0
    }
    difference_in_percent = 1 - (get_mde(**params, metric='cuped_revenue') /
                                 get_mde(**params, metric='revenue_amount'))
    result_dict[(expr_start, pretest_start, pretest_end)] = round(difference_in_percent * 100, 1)
for (expr, pretest_start_date, pretest_end_date) in result_dict:
    print(f'Претест идет {pretest_start_date} - {pretest_end_date} и тест начинается {expr} и длится {CONST_SHIFT_DAYS} дней:')
    print(f'MDE уменьшился на {result_dict[(expr, pretest_start_date, pretest_end_date)]}%!')

Претест идет 2019-10-09 - 2019-11-09 и тест начинается 2019-11-09 и длится 30 дней:
MDE уменьшился на 53.8%!
Претест идет 2019-09-09 - 2019-10-10 и тест начинается 2019-10-10 и длится 30 дней:
MDE уменьшился на 48.0%!
Претест идет 2019-08-10 - 2019-09-10 и тест начинается 2019-09-10 и длится 30 дней:
MDE уменьшился на 35.4%!
Претест идет 2019-07-11 - 2019-08-11 и тест начинается 2019-08-11 и длится 30 дней:
MDE уменьшился на 39.1%!
Претест идет 2019-06-11 - 2019-07-12 и тест начинается 2019-07-12 и длится 30 дней:
MDE уменьшился на 43.5%!


Отмечу, что в логах пишется с невключенной правой границей претеста, чтобы не было ликов в данных.
Если брать константные окна в 30 дней (тест тоже двигаем), то видим, что разное, если брать различные месяцы претеста, это объясняется наличием некоторой сезонности. Попробуем не сдвигать окно теста.

In [273]:
result_dict = {}
for offset in [0, 30, 60, 90, 120]:
    df_expr_offset, expr_start, pretest_start, pretest_end  = preprocess_data(df,
                                                                              offset=offset,
                                                                              pre_test_offset=0,
                                                                              move_test=False
                                                                             )
    params = {
        'data': df_expr_offset,
        'share': 0.5,
        'shift_days': CONST_SHIFT_DAYS,
        'alpha': 0.01,
        'power': 0.8,
        'offset': 0
    }
    difference_in_percent = 1 - (get_mde(**params, metric='cuped_revenue') /
                                 get_mde(**params, metric='revenue_amount'))
    result_dict[(expr_start, pretest_start, pretest_end)] = round(difference_in_percent * 100, 1)
for (expr, pretest_start_date, pretest_end_date) in result_dict:
    print(f'Претест идет {pretest_start_date} - {pretest_end_date} и тест начинается {expr} и длится {CONST_SHIFT_DAYS} дней:')
    print(f'MDE уменьшился на {result_dict[(expr, pretest_start_date, pretest_end_date)]}%!')
    print('='*75)

Претест идет 2019-10-09 - 2019-11-09 и тест начинается 2019-11-09 и длится 30 дней:
MDE уменьшился на 53.8%!
Претест идет 2019-09-09 - 2019-10-10 и тест начинается 2019-11-09 и длится 30 дней:
MDE уменьшился на 43.8%!
Претест идет 2019-08-10 - 2019-09-10 и тест начинается 2019-11-09 и длится 30 дней:
MDE уменьшился на 31.6%!
Претест идет 2019-07-11 - 2019-08-11 и тест начинается 2019-11-09 и длится 30 дней:
MDE уменьшился на 27.2%!
Претест идет 2019-06-11 - 2019-07-12 и тест начинается 2019-11-09 и длится 30 дней:
MDE уменьшился на 27.8%!


Собственно, также уменьшается эффект от CUPED, пользователи по-разному ведут себя все же в интервалы - сезонность, поэтому корреляция между агрегатом пре-метрики ниже с нашей целевой метрикой, поэтому и уменьшение дисперсии ниже => MDE уменьшается меньше. 

Попробуем теперь удлинить сбор пре-теста, то есть брать не константно 30 дней, а увеличим его, возможно сможем сгладить сезонность.

In [274]:
result_dict = {}
for offset in [0, 30, 60, 90]:
    for pre_test_offset_var in range(30, 150-offset, 30):
        df_expr_offset, expr_start, pretest_start, pretest_end  = preprocess_data(df,
                                                                              offset=offset,
                                                                              pre_test_offset=pre_test_offset_var,
                                                                              move_test=False
                                                                             )
        params = {
            'data': df_expr_offset,
            'share': 0.5,
            'shift_days': CONST_SHIFT_DAYS,
            'alpha': 0.01,
            'power': 0.8,
            'offset': 0
        }
        difference_in_percent = 1 - (get_mde(**params, metric='cuped_revenue') /
                                     get_mde(**params, metric='revenue_amount'))
        result_dict[(expr_start, pretest_start, pretest_end)] = round(difference_in_percent * 100, 1)
for (expr, pretest_start_date, pretest_end_date) in result_dict:
    print(f'Претест идет {pretest_start_date} - {pretest_end_date} и тест начинается {expr} и длится {CONST_SHIFT_DAYS} дней:')
    print(f'MDE уменьшился на {result_dict[(expr, pretest_start_date, pretest_end_date)]}%!')
    print('='*75)

Претест идет 2019-09-09 - 2019-11-09 и тест начинается 2019-11-09 и длится 30 дней:
MDE уменьшился на 53.9%!
Претест идет 2019-08-10 - 2019-11-09 и тест начинается 2019-11-09 и длится 30 дней:
MDE уменьшился на 51.0%!
Претест идет 2019-07-11 - 2019-11-09 и тест начинается 2019-11-09 и длится 30 дней:
MDE уменьшился на 50.5%!
Претест идет 2019-06-11 - 2019-11-09 и тест начинается 2019-11-09 и длится 30 дней:
MDE уменьшился на 50.3%!
Претест идет 2019-08-10 - 2019-10-10 и тест начинается 2019-11-09 и длится 30 дней:
MDE уменьшился на 41.0%!
Претест идет 2019-07-11 - 2019-10-10 и тест начинается 2019-11-09 и длится 30 дней:
MDE уменьшился на 41.1%!
Претест идет 2019-06-11 - 2019-10-10 и тест начинается 2019-11-09 и длится 30 дней:
MDE уменьшился на 41.2%!
Претест идет 2019-07-11 - 2019-09-10 и тест начинается 2019-11-09 и длится 30 дней:
MDE уменьшился на 33.1%!
Претест идет 2019-06-11 - 2019-09-10 и тест начинается 2019-11-09 и длится 30 дней:
MDE уменьшился на 33.5%!
Претест идет 2019-0

Окей, максимум уменьшения, если претест имеет максимальный размер. Попробуем теперь брать эксперимент не 30 дней, а другое количество дней.

In [275]:
for duration in [60, 90]:
    array_of_test_ranges = None
    if duration == 90:
        array_of_test_ranges = [-30, 0]
    else:
        array_of_test_ranges = [-30, 0, 30, 60]
    for test_range_sub in array_of_test_ranges:
        
        df_expr_offset, expr_start, pretest_start, pretest_end  = preprocess_data(df,
                                                                                offset=0,
                                                                                pre_test_offset=test_range_sub,
                                                                                move_test=False,
                                                                                test_duration=duration
                                                                                )
        params = {
            'data': df_expr_offset,
            'share': 0.5,
            'shift_days': 90,
            'alpha': 0.01,
            'power': 0.8,
            'offset': 0
        }
        difference_in_percent = 1- (get_mde(**params, metric='cuped_revenue') /
                                   get_mde(**params, metric='revenue_amount'))
        print(f'Претест идет {pretest_start} - {pretest_end} и тест начинается {expr_start} и длится {duration} дней:')
        print(f'MDE уменьшился на {difference_in_percent * 100:.2f}%!')
        print('='*75)

Претест идет 2019-09-09 - 2019-10-10 и тест начинается 2019-10-10 и длится 60 дней:
MDE уменьшился на 45.15%!
Претест идет 2019-08-10 - 2019-10-10 и тест начинается 2019-10-10 и длится 60 дней:
MDE уменьшился на 46.10%!
Претест идет 2019-07-11 - 2019-10-10 и тест начинается 2019-10-10 и длится 60 дней:
MDE уменьшился на 45.91%!
Претест идет 2019-06-11 - 2019-10-10 и тест начинается 2019-10-10 и длится 60 дней:
MDE уменьшился на 47.00%!
Претест идет 2019-07-11 - 2019-09-10 и тест начинается 2019-09-10 и длится 90 дней:
MDE уменьшился на 41.23%!
Претест идет 2019-06-11 - 2019-09-10 и тест начинается 2019-09-10 и длится 90 дней:
MDE уменьшился на 41.07%!


CUPED работает хуже, чем для длины теста в 30 дней.

Максимум наблюдается при длине теста 30 дней и длине пре-теста 90 дней.

In [281]:
df_expr_offset, expr_start, pretest_start, pretest_end  = preprocess_data(df,
                                                                            offset=0,
                                                                            pre_test_offset=30,
                                                                            move_test=False,
                                                                            test_duration=30
                                                                            )
params = {
    'data': df_expr_offset,
    'share': 0.5,
    'shift_days': 90,
    'alpha': 0.01,
    'power': 0.8,
    'offset': 0
}
difference_in_percent = 1- (get_mde(**params, metric='cuped_revenue') /
                           get_mde(**params, metric='revenue_amount'))
print(f'Претест идет {pretest_start} - {pretest_end} и тест начинается {expr_start} и длится {30} дней:')
print(f'MDE уменьшился на {difference_in_percent * 100:.2f}%!')
print('='*75)

Претест идет 2019-09-09 - 2019-11-09 и тест начинается 2019-11-09 и длится 30 дней:
MDE уменьшился на 53.87%!


И максимум наблюдается при длине пре-теста 60 дней и длины самого эксперимента 30 дней.

## Стратификация

Необходимо решить, на какие бакеты бить пользователей, можно попробовать просто перебором через различные группировки. Так как у нас зафиксированы в целом гиперпараметры в виде длины теста и доли пользователей в тесте, то можно просто перебрать в цикле группировки и решить, где будет максимум.

In [282]:
filter_test = (df['event_date'] >= pd.to_datetime('2019-11-09')) &\
              (df['event_date'] <= pd.to_datetime('2019-12-09'))

In [283]:
df_test_filtered = df[filter_test]

In [364]:
cat_features = ['user_segment', 'category', 'region',
                ['user_segment', 'region'],
               ['user_segment', 'category']]

Некоторые фичи можно группировать попарно, тем самым делая количество страт больше, то есть регион с user_segment выдаст около 200 страт, что как раз то, что нужно. Попробуем одномерное группирование, затем подключим двумерное для некоторых. Очевидно, что группировать по категории и региону мы не будем - так будет группы, в которых малое количество объектов.

In [285]:
df_test_filtered.head()

,event_date,user_id,user_segment,category,region,revenue_amount
6143766,2019-11-09,12310733178758519,2376074197230840906,1472231361867825698,5780543780372929118,126
6143767,2019-11-09,13486212324750086,2376074197230840906,1560592244484430230,4848067518890897757,697
6143768,2019-11-09,24616357645798696,2376074197230840906,57514010642945023,7126065072605124185,705
6143769,2019-11-09,28775360471736672,4626944681007198896,7247164925237372155,2751451541457411742,109
6143770,2019-11-09,37539528096930720,4626944681007198896,57514010642945023,3048517866726460689,193


In [367]:
for col in cat_features:
    df_copy_test = df_test_filtered.copy()
    max_date = df_copy_test['event_date'].max()
    df_copy_test = choose_date(df_copy_test, max_date, 30, 0.5, 0)

    strata_p = pd.DataFrame(df_copy_test.groupby(col).count()['revenue_amount']/df_copy_test.shape[0]).reset_index()
    strata_p.rename(columns={'revenue_amount': 'proba'},
                   inplace=True)
    mean = df_copy_test['revenue_amount'].mean()
    std_prev = df_copy_test['revenue_amount'].std()
    df_copy_test = df_copy_test.merge(strata_p, on=col, how='left')

    std = (df_copy_test.groupby(col).var().revenue_amount*df_copy_test.groupby(col).mean().proba).sum() ** 0.5
    effect_size = tt_ind_solve_power(effect_size=None, alpha=0.01, power=0.8, nobs1=df_copy_test.shape[0],
                                    ratio=1, alternative = 'two-sided')
    
    print(f'Для сегмента {col}:')
    strat_mde = effect_size * (std / mean) * 100
    mde = effect_size * (std_prev / mean) * 100
    print(f'MDE = {strat_mde:.1f}%')
    print(f'MDE без стратификации = {mde:.1f}%')
    diff = 1 - strat_mde / mde
    print(f'Относительная разница {diff * 100:.1f}%')
    print('='*75)

Для сегмента user_segment:
MDE = 4.2%
MDE без стратификации = 5.5%
Относительная разница 24.3%
Для сегмента category:
MDE = 3.5%
MDE без стратификации = 5.5%
Относительная разница 36.1%
Для сегмента region:
MDE = 3.6%
MDE без стратификации = 5.5%
Относительная разница 35.3%
Для сегмента ['user_segment', 'region']:
MDE = 2.7%
MDE без стратификации = 5.5%
Относительная разница 50.8%
Для сегмента ['user_segment', 'category']:
MDE = 3.3%
MDE без стратификации = 5.5%
Относительная разница 39.6%


Получается, лучший эффект, если стратифицировать по user_segment и region, почти сопоставим с результатом от cuped.